#### Importing Libraries

In [62]:
import datetime
import json
from bs4 import BeautifulSoup
import requests
from dataclasses import dataclass

#### Making the Holiday class

In [137]:
class Holiday:
    def __init__(self, name, date):
            
        date = datetime.datetime.strptime(date, '%Y-%m-%d')
        
        self._name = name 
        self._date = date
        self.holiday_dict = {}
    
    def __str__(self):
        return f'The Holiday is {self._name}, which occurs on {self._date}.'
    
    def __dict__(self, name, date):
        self.holiday_dict[name] = date
    
    def __showdict__(self):
        return self.holiday_dict

Christmas_maker = Holiday('Christmas', '1999-12-25')
Christmas_var = vars(Christmas_maker.__showdict__)
print(Christmas_var)
print(Christmas_maker)

{}
The Holiday is Christmas, which occurs on 1999-12-25 00:00:00.


In [117]:
class HolidayList:
    def __init__(self, holidayObj):
        self.innerHolidays = []
        self.holidayObj = holidayObj
        #else: raise Exception()

    def addHoliday(self, holidayObj):
        
        self.innerHolidays.append(holidayObj.__dict__)
        return self.innerHolidays

    def __str__(self):
        return f'The Holidays are as follows: {self.innerHolidays}'
    
    def __repr__(self):
        return self.innerHolidays
    
    def __list__(self):
        return self.innerHolidays
    
    def findHoliday(HolidayName, Date):
        return HolidayName, Date

full_list = HolidayList(Christmas)
print(full_list)

<bound method Holiday.__dict__ of <__main__.Holiday object at 0x000001937B208550>>
The Holidays are as follows: []
